In [ ]:
!pip install onnx onnxruntime transformers txtai

In [ ]:
from txtai.pipeline import HFOnnx

onnx = HFOnnx()

model = onnx(
    "intfloat/multilingual-e5-large",
    "pooling",
    "/tmp/e5.onnx"
)

In [ ]:
!mkdir /tmp/compacted

In [ ]:
import onnx

onnx_model = onnx.load("/tmp/e5.onnx")

onnx.save_model(
    onnx_model,
    "/tmp/compacted/e5.onnx",
    save_as_external_data=True,
    all_tensors_to_one_file=True,
    location="e5.onnx_data",
    size_threshold=1024,
    convert_attribute=False
)


In [ ]:
!ls -lh /tmp/compacted

total 4.2G
-rw-r--r-- 1 root root 590K Feb 19 13:45 e5.onnx
-rw-r--r-- 1 root root 4.2G Feb 19 13:45 e5.onnx_data


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
tokenizer.save_pretrained("/tmp/tokenizer")

('/tmp/tokenizer/tokenizer_config.json',
 '/tmp/tokenizer/special_tokens_map.json',
 '/tmp/tokenizer/sentencepiece.bpe.model',
 '/tmp/tokenizer/added_tokens.json',
 '/tmp/tokenizer/tokenizer.json')

In [ ]:
from onnxruntime import InferenceSession
from transformers import AutoTokenizer

session = InferenceSession(
    "/tmp/compacted/e5.onnx",
    providers=['CPUExecutionProvider']
)

tokenizer = AutoTokenizer.from_pretrained("/tmp/tokenizer")

In [ ]:
input = tokenizer(
    "I am happy. I am glad to help. This is another, a bit longer sentence.",
    return_tensors="np"
)

output = session.run(None, dict(input))

In [ ]:
print(output)

[array([[ 0.45499966, -1.0945951 , -0.37001383, ..., -0.8371148 ,
        -0.6760583 , -0.01711324]], dtype=float32)]


In [ ]:
type(output[0][0])

numpy.ndarray

In [ ]:
print(len(output[0][0]))

1024
